# Upload Spatial Data to sandbox
### Match format to existing table in  main portion of database
Note: geometry information must be in columns labeled "lat" and "long"; geometry is created in column "geom"

In [1]:
import sys
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
sys.path.append("C:/Users/ehbaker/Documents/Python/Repos/ice2O") #Path to where DBImport.py is saved
import DbImport #User defined module in the folder added to path in line above.
import numpy as np
import settings

### User supplied criteria

In [3]:
#For snow radar
db_table='snowradar' #name of table in the database which you want to copy
pth=(r"Q:\Project Data\GlacierData\GPR\Wolverine\2016\Ice2ODatabase\Wolverine_2016.csv") #path to csv for upload

Connect to database and sandbox:

In [4]:
cs=settings.import_cs() #user-defined module to store connection info
#Spatial_database
engine = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + cs['dbname'])
#Sandbox
engine_sand = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + 'sandbox')

Read in csv for upload:

In [5]:
#Read in csv for upload
df=pd.read_csv(pth) #read to dataframe
df=df.sample(30)
#Extract format from existing table in main database
types=DbImport.define_db_table_format(df, db_table, engine)

Column names and Posgres data types for existing database table:

In [6]:
#Show the dataframe and the column types
print(types) #data in DB
df[0:2]#data in table

     attname                  type
0       elev      double precision
1       twtt      double precision
2  thickness      double precision
3        swe      double precision
4      trace               integer
5       geom  geometry(Point,3338)


,trace,long,lat,elev,twtt,thickness,swe,collection
229555,7088,-148.914357,60.415756,1254.65,10.11,1.092,0.502,WOLVERINE_2016_D4_LINE08
75925,8991,-148.931324,60.425762,1357.35,82.00,8.856,4.074,WOLVERINE_2016_D1_LINE16


Change the order of columns in table to match that in database

In [7]:
#Remove columns in table that should be uploaded to the database (for snowradar, that is removing collection)
cols_to_keep=list(set(list(df)) -(set(list(df)) -set(list(types['attname'])+ [u'long', u'lat'])))
#Remove unwanted columns
df=df[cols_to_keep]
#Reorder columns to match order in DB, with the addition of lat/long, WITHOUT the geom column (will be created in SQL)
df=df[list(types[~types.attname.str.contains("geom")]['attname'])+[u'long', u'lat']].copy() #copy neccesary to overwrite
df[0:2]

,elev,twtt,thickness,swe,trace,long,lat
229555,1254.65,10.11,1.092,0.502,7088,-148.914357,60.415756
75925,1357.35,82.00,8.856,4.074,8991,-148.931324,60.425762


### Check for primary key, and if exists as numeric, add column for pkey in ingested table and continue numbering
A better workflow may be to simply delete primary key and re-create with each upload. Currently the p-key as implemented in the database does not serve as a foreign key, and does not relate to any meaningful variable; rather, simple identifier.

In [8]:
#If numeric, add IDs sequentially, starting +1 from the current max
res=DbImport.pkey_NameAndType(db_table, engine)
if res=='None':
    print("No need to add primary key")
else:
    pkey=res['attname'][0]
    pkey_type=res['data_type'][0]
    if pkey_type in ['smallint', 'integer', 'bigint', 'decimal', 'numeric', 'real', 'double precision', 'smallserial', 'serial', 'bigserial']:
        print ("Primary Key = Numeric \nAdding the primary key and unique IDs to rows of table being appended")
        df=DbImport.add_sequential_IDs_to_pkey(df, db_table, engine) #overwrite table
    else:
        print("Primary ID is not Numeric; must be updated manually")

PLEASE NOTE: Table has NO primary key
No need to add primary key


In [9]:
#Add the subset of data (df) to sandbox
dbnamePts = db_table +'_ingest'
df.to_sql(dbnamePts, engine_sand, index = False, if_exists='replace')

In [10]:
#Create geomoetry field:
engine_sand.execute("""ALTER TABLE %s ADD COLUMN geom geometry(Point, 3338);""" %(dbnamePts)) 
# populate the geometry field
engine_sand.execute("""UPDATE %s SET geom = ST_Transform(ST_setSRID(ST_MakePoint(long,lat),4326),3338);""" %(dbnamePts))

In [11]:
#Reformat columns in sandbox to match format of those in existing table in spatial_database
for xx in range(0,types.shape[0]):
    col_name=types.attname[xx]
    col_type=types.type[xx]
    query=(r"""ALTER TABLE %s
     ALTER COLUMN %s TYPE %s
     USING %s::%s""")%(dbnamePts, col_name, col_type, col_name, col_type)
    engine_sand.execute(query)
    print("done with " +col_name)
print("DONE with" + dbnamePts)

done with elev
done with twtt
done with thickness
done with swe
done with trace
done with geom
DONE withsnowradar_ingest


In [18]:
#Remove Lat and Long columns
engine_sand.execute("ALTER TABLE %s DROP COLUMN IF EXISTS %s"%(dbnamePts, 'lat'))
engine_sand.execute("ALTER TABLE %s DROP COLUMN IF EXISTS %s"%(dbnamePts, 'long'))

In [12]:
#Set owner to Administrator
engine_sand.execute("ALTER TABLE %s OWNER TO administrator"%(dbnamePts))

In [13]:
#Set permissions on tables. 
###These are currently not working, as passed thru SQLAlchemy engine; functional when passed direcly in SQL window in PgAdmin.
engine_sand.execute("GRANT SELECT ON TABLE %s TO reader;"%(dbnamePts))
engine_sand.execute("GRANT ALL ON TABLE %s TO administrator"%(dbnamePts))
engine_sand.execute("ALTER TABLE %s OWNER TO administrator;"%(dbnamePts))

### Done! You have uploaded a table to the sandbox which matches structure of the original table (columns and types)